In [ ]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key="sk-65e478f6033944a28ba38274d6785f07", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

In [1]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import sys
import os

c:\Users\ALTjasonspeed\.conda\envs\JJAZ\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from prompts.temp_expert_prompt import e_system_prompt, e_user_prompt
from prompts.temp_prompt import system_prompt, user_prompt

def generate_expert(dilema_decription, expert_opionion):
    client = OpenAI(api_key="sk-65e478f6033944a28ba38274d6785f07", base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": f"{e_system_prompt}"},
            {"role": "user", "content": f"{e_user_prompt.format(dilema_decription, expert_opionion)}"},
        ],
        stream=False,
        temperature=0,
    )
    
    return response.choices[0].message.content

def generate_output(dilemma_decription):
    client = OpenAI(api_key="sk-65e478f6033944a28ba38274d6785f07", base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": f"{system_prompt}"},
            {"role": "user", "content": f"{user_prompt.format(dilemma_decription)}"},
        ],
        stream=False,
        temperature=0,
    )
    
    return response.choices[0].message.content

In [3]:
llm_answers = pd.read_csv("../data/llm_answers.csv")
llm_answers = llm_answers[["Dilemma Description", "Expert Opinion"]]
llm_answers["expert_output"] = llm_answers.apply(lambda row: generate_expert(row['Dilemma Description'], row['Expert Opinion']), axis=1)
llm_answers["dilemma_output"] = llm_answers.apply(lambda row: generate_output(row['Dilemma Description']), axis=1)

temp = llm_answers['expert_output'].str.split('%', expand=True)
temp.columns = ['' if col == 0 else temp[col].str.split(':', expand=True)[0].iloc[0] for col in temp.columns]
for col in temp.columns[1:]:
    llm_answers["Expert " + col] = temp[col].str.split(':', expand=True)[1].str.strip()

temp2 = llm_answers['dilemma_output'].str.split('%', expand=True)
temp2.columns = ['' if col == 0 else temp2[col].str.split(':', expand=True)[0].iloc[0] for col in temp2.columns]
for col in temp2.columns[1:]:
    llm_answers["Dilemma " + col] = temp2[col].str.split(':', expand=True)[1].str.strip()

llm_answers.to_csv("deepseek_llm_answers_structured_output.csv")